# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

El caso de uso que se busca cubrir hace referencia a detectar si una jugada termina en home run o no, cuando la bola ya fue bateada. Solo se va a poder usar dicho modelo para predecir el resultado de una jugada si se tienen los datos del lanzamiento y el bateo. En consecuencia, podemos decir que su principal uso puede relacionarse con el estudio y análisis de las mejores formas para realizar un home run

A partir del dataset elegido y ya preprocesado con la lógica definida en el TP1, el grupo debe realizar el entrenamiento y evaluación de al menos 3 algoritmos de machine learning.

   Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.
   Se debe aplicar alguna técnica de feature engineering para mejorar los datos de entrada a los modelos, y mostrar la comparativa de los resultados obtenidos en cada caso. Si no es posible o útil, fundamentar el motivo por el cual no se realizará.
   Por cada modelo, se debe entrenarlo y realizar una exploración de hiper-parámetros mediante una búsqueda en grilla. Evaluar el comportamiento de cada modelo con los hiper-parámetros que mejores resultados ofrecen. En caso de ser posible, aporte conclusiones respecto a dicha comparación.
   Realizar experimentos que utilicen como datos de entrada representaciones intermedias de los datos (generadas por técnicas de reducción de dimensiones como PCA). Compare los resultados obtenidos contra los casos previos, interprete y proponga conclusiones.
   Se deben utilizar técnicas que garanticen que los modelos no están sobreentrenando sin que nos demos cuenta.
   Determinar el valor final de la métrica que podría ser informado al cliente, utilizando técnicas que permitan obtener un valor lo más realista posible. Fundamentar y considerar no solo el rendimiento del modelo en su elección, sino también cuestiones como interpretabilidad, tiempos de entrenamiento, etc.
   Para el método propuesto como definitivo, y para distintos pares de variables, genere diagramas de dispersión donde se visualicen los aciertos y errores del mismo. Discuta si existen patrones o conocimiento que se pueda obtener a partir de dichos errores. En caso de ser posible, evalúe la importancia que asigna el método a las variables de entrada y genere conclusiones al respecto.


## Tratamiento de datos aplicando en el TPN°1

In [1]:
# Importamos las dependencias necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns
import warnings
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelBinarizer
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import QuantileTransformer
warnings.filterwarnings('ignore')

In [2]:
#Arreglamos el dataset según lo establecido en el TP1

# Importamos train.csv y park_dimensions.csv, los unimos utilizando la variable "park"
entrenamiento = pd.read_csv('./train.csv')
estadio = pd.read_csv('./park_dimensions.csv')
completa=entrenamiento.merge(estadio, on="park", how="left")

# Desechamos las variables no utilizadas
completa = completa.drop(['park','bip_id','batter_id','pitcher_id'],axis=1)

# Asignamos nuevos nombres a las columnas
renamed_columns = {'NAME': 'name', 'Cover': 'cover', 'LF_Dim': 'lf_dim', 'CF_Dim':'cf_dim',
                   'RF_Dim': 'rf_dim', 'LF_W': 'lf_w', 'CF_W': 'cf_w', 'RF_W': 'rf_w'
                  }
completa.rename(columns=renamed_columns, inplace=True)

# Convertir columna "game_date" de tipo object/string, a datetime
completa['game_date'] = pd.to_datetime(completa['game_date'])

# Eliminar datos filas con datos nulos en bb_type
completa = completa[~completa.bb_type.isnull()]
#Delimitación de conjuntos
completa.isnull().sum()


east = ['TB','BAL','BOS','TOR','NYY','ATL','MIA','PHI','NYM','WSH']
central = ['MIN','CLE','DET','CWS','KC','PIT','MIL','CHC', 'CIN', 'STL']
west = ['TEX','LAA','HOU','SEA','OAK','LAD','SD','SF','COL','ARI']

def division_h(row):
    if east.count(row['home_team']) > 0:
        return 'east'
    else:
        if central.count(row['home_team']) > 0:
            return 'central'
        else:
            return 'west'
        
def division_a(row):
    if east.count(row['away_team']) > 0:
        return 'east'
    else:
        if central.count(row['away_team']) > 0:
            return 'central'
        else:
            return 'west'
        
division_home = completa.apply(division_h, axis=1)
division_away = completa.apply(division_a, axis=1)
completa["division_home"] = division_home
completa["division_away"] = division_away
completa.sample(10)

columnas_string=['game_date','home_team','away_team','batter_team','batter_name','pitcher_name','name']
completa = completa.drop(columnas_string,axis=1)
#completa[["division_away",'away_team']]

## Selección de métrica

Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.

Con respecto a las métricas, podemos decir que vamos a utilizar recall y precisión. Esto se debe a que los datos están muy desbalanceados, y en consecuencia accuracy no sería la mejor opción porque presentaría problemas.

 - Recall nos va a permitir ver de todos los Foo que había, cuántos encontramos.

 - Precisión que nos va a permitir ver de lo que clasificamos como Foo, qué porcentaje era realmente Foo.

## Aplicaciones de featuring engineering

En lo referente a feature enginering hay que hay que mencionar que vamos a aplicar la técnica de QuantileTransformer para las variables de entrada que tienen valores extremos según lo observado en el TP1. De esta forma se logra reducir el impacto de los valores atípicos. También vamos a transformar algunas variables para transformar la información de las mismas expresada de otra forma. 
Aplicamos técnicas de preprocesado para mejorar la representación de los datos como OneHotEncoder y StandardImputer, y eliminamos los valores nulos utilizando SimpleImputer.

### División del dataset.

El dataset se va a dividir en tres grupos:
 - Train (60%)
 - Test (20%)
 - Validation (20%)

In [4]:
#División de datasets en "train", "test" y "validation"
from sklearn.model_selection import train_test_split

train, not_train = train_test_split(completa, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

((27742, 24), (9248, 24), (9248, 24))

### Aplicación de DataFrameMapper

In [76]:
# Definimos el mapper. Recibe una lista de (columna/s, transformers)

mapper = DataFrameMapper([ 
    #(['is_batter_lefty'], None),
    #(['is_pitcher_lefty'], None),
    #(['bb_type'], [OneHotEncoder()]),
    #(['bearing'], [OneHotEncoder()]),
    #(['pitch_name'], [OneHotEncoder()]),
    #(['inning'], [StandardScaler()]),
    #(['outs_when_up'], None),
    #(['balls'], [StandardScaler()]),
    #(['strikes'], [MinMaxScaler()]),
    #(['plate_x'], [MinMaxScaler()]),
    #(['plate_z'], [MinMaxScaler()]),
    (['pitch_mph'], [StandardScaler()]),
    (['launch_speed'], [StandardScaler()]),
    (['launch_angle'], [StandardScaler()]),
    (['is_home_run'], None),
    #(['cover'], [OneHotEncoder()]),
    (['lf_dim'], [MinMaxScaler()]),
    (['cf_dim'], [MinMaxScaler()]),
    (['rf_dim'], [MinMaxScaler()]),
    (['lf_w'], [MinMaxScaler()]),
    (['cf_w'], [MinMaxScaler()]),
    (['rf_w'], [MinMaxScaler()]),
    (["division_home"], [OneHotEncoder()]),
    (["division_away"], [OneHotEncoder()]),
])

mapper.fit(train)

#completa_processed = mapper.transform(completa)
#completa_processed = pd.DataFrame(completa_processed, columns=mapper.transformed_names_)
#completa_processed
#mapper.transformed_names_
#completa_processed.isnull().sum()

DataFrameMapper(drop_cols=[],
                features=[(['pitch_mph'], [StandardScaler()]),
                          (['launch_speed'], [StandardScaler()]),
                          (['launch_angle'], [StandardScaler()]),
                          (['is_home_run'], None),
                          (['lf_dim'], [MinMaxScaler()]),
                          (['cf_dim'], [MinMaxScaler()]),
                          (['rf_dim'], [MinMaxScaler()]),
                          (['lf_w'], [MinMaxScaler()]),
                          (['cf_w'], [MinMaxScaler()]),
                          (['rf_w'], [MinMaxScaler()]),
                          (['division_home'], [OneHotEncoder()]),
                          (['division_away'], [OneHotEncoder()])])

### Eliminar nulos y aplicar Dataframe

In [ ]:
#Combinación de lo anterior

pipe1 = Pipeline([
    ('mapper', mapper),
    ('imputer', SimpleImputer(strategy='mean')),
])
# Lo entrenamos con train
pipe1.fit(completa)
completa_com = pipe1.transform(completa)
completa_com=pd.DataFrame(completa_com, columns=mapper.transformed_names_)
completa_com.isnull().sum()
#train_b
#mapper.transformed_names_

### Aplicación de QuantileTransformer

Según lo visto en el TP1, consideramos necesario aplicar dicha técnica a los variables "plate_x", "plate_z", "pitch_mph", "launch_speed" y "launch_angle". Esto se debe a que contienen valores extremos que pueden traer problemas para la predicción.

In [ ]:
def plot_histogram_transformation(columna):

    fig, axis = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

    x = columna
    x.hist(ax=axis[0], bins=20)
    axis[0].set_title('Distribución original')
    
    tr = QuantileTransformer(n_quantiles=50, output_distribution='normal')
    tr.fit(x.to_frame())

    x2 = tr.transform(x.to_frame())
    pd.Series(x2.flatten()).hist(ax=axis[1], bins=20)
    axis[1].set_title('Distribución normal')
    return x2

#plot_histogram_transformation()
#completa['pitch_mph']= plot_histogram_transformation(completa.plate_x)
#completa['pitch_mph']
#completa['pitch_mph']= plot_histogram_transformation(completa.plate_z)
#completa['pitch_mph']
completa['pitch_mph']= plot_histogram_transformation(completa.pitch_mph)
#completa['pitch_mph']
#completa['launch_speed']= plot_histogram_transformation(completa.launch_speed)
#completa['pitch_mph']
#completa['pitch_mph']= plot_histogram_transformation(completa.launch_angle)
#completa['pitch_mph']

## Modelos a utilizar

Los modelos que vamos a utilizar van a ser:
 - LogisticRegression
 - KNeighborsClassifier
 - GridSearchCV
 - RandomForestClassifier

In [82]:
from sklearn import metrics

pipel = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', LogisticRegression(random_state=42)),
])

pipel.fit(train, train.is_home_run)

y_pred = pipel.predict(test)
y_pred
print(metrics.classification_report(test.is_home_run, y_pred))
metrics.precision_score(test.is_home_run, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8759
           1       1.00      1.00      1.00       489

    accuracy                           1.00      9248
   macro avg       1.00      1.00      1.00      9248
weighted avg       1.00      1.00      1.00      9248



1.0

In [78]:
K=200
knn_model = Pipeline([
    ('mapper', mapper),
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', KNeighborsClassifier(n_neighbors=K)),
])

knn_model.fit(train, train.is_home_run)
y_pred = knn_model.predict(validation)
y_pred
print(metrics.classification_report(validation.is_home_run, y_pred))


metrics.recall_score(validation.is_home_run, y_pred)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8795
           1       1.00      0.78      0.87       453

    accuracy                           0.99      9248
   macro avg       0.99      0.89      0.93      9248
weighted avg       0.99      0.99      0.99      9248



0.7770419426048565

In [91]:


forest_model_limits= RandomForestClassifier(n_estimators=100, max_depth=3, max_features=3, random_state=42)
# n_estimators? max_depth=3?, max_features=2?

rf_model = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', forest_model_limits),
])

rf_model.fit(train, train.is_home_run)

y_pred = rf_model.predict(validation)
y_pred
print(metrics.classification_report(validation.is_home_run, y_pred))


metrics.recall_score(validation.is_home_run, y_pred)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8795
           1       1.00      0.89      0.94       453

    accuracy                           0.99      9248
   macro avg       1.00      0.95      0.97      9248
weighted avg       0.99      0.99      0.99      9248



0.8940397350993378

In [68]:
#Grid Search:
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors': [100, 2, 50]}

clf = GridSearchCV(KNeighborsClassifier, parameters, refit=True, verbose=1)

gs_pipe = Pipeline([
    ('mapper', mapper),
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', clf),
])

gs_pipe.fit(train, train.is_home_run)
gs_pipe.predict(validation)
#clf.best_score_, clf.best_params_

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

Por cada modelo, se debe entrenarlo y realizar una exploración de hiper-parámetros mediante una búsqueda en grilla. Evaluar el comportamiento de cada modelo con los hiper-parámetros que mejores resultados ofrecen. En caso de ser posible, aporte conclusiones respecto a dicha comparación.

## Técnicas de reducción de la dimensionalidad 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
pca_pipe = make_pipeline(StandardScaler(), PCA())
pca_pipe.fit(train_b)

modelo_pca = pca_pipe.named_steps['pca']
print(train_b.columns)

print('----------------------------------------------------')
print('Porcentaje de varianza explicada por cada componente')
print('----------------------------------------------------')
print(modelo_pca.explained_variance_ratio_)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax.bar(
    x      = np.arange(modelo_pca.n_components_) + 1,
    height = modelo_pca.explained_variance_ratio_
)

for x, y in zip(np.arange(len(train_b.columns)) + 1, modelo_pca.explained_variance_ratio_):
    label = round(y, 2)
    ax.annotate(
        label,
        (x,y),
        textcoords="offset points",
        xytext=(0,10),
        ha='center'
    )

ax.set_xticks(np.arange(modelo_pca.n_components_) + 1)
ax.set_ylim(0, 1.1)
ax.set_title('Porcentaje de varianza explicada por cada componente')
ax.set_xlabel('Componente principal')
ax.set_ylabel('Por. varianza explicada');
#train_processed.head=X_cols

#pca= PCA(n_components=4,random_state=42)
#train= pca.fit_transform(train_b)
#[:37]